In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [2]:
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
import pandas as pd
import bs4
import requests
from urllib.request import urlopen

In [4]:
# 각 섹션별 링크주소 파일 가져오기
df_menu = pd.read_csv('./crawl_data/naver_news_section.csv',index_col=0)
df_menu.head()

,section,link
0,정치,https://news.naver.com/main/main.naver?mode=LS...
1,경제,https://news.naver.com/main/main.naver?mode=LS...
2,사회,https://news.naver.com/main/main.naver?mode=LS...
3,생활/문화,https://news.naver.com/main/main.naver?mode=LS...
4,IT/과학,https://news.naver.com/main/main.naver?mode=LS...


### 기존 코드(02_1) 내용을 결합해서 함수로 구성
- 다른 섹션에도 적용하기 위해
- 함수명 : get_topic(url,section)
- 반환 : 섹션별 수집 데이터를 dict로 구성 후 반환
- 함수가 반환해 준 dict를 df로 구성 후에 기존 df에 모두 결합해서 df 완성
    - 6개의 섹션의 헤드라인을 한 파일로 구성

In [5]:
# 함수 구성
def get_topic(url, section) :
    # 전달된 url의 소스 요청하는 코드
    res = urlopen(url)
    html = res.read()
    # 요청 후 반환된 소스 bs객체로 생성
    bs_obj = bs4.BeautifulSoup(html, 'html.parser')
    # headline 뉴스 전체 추출
    # class : sh_text
    head_line = bs_obj.findAll('div',{'class':'sh_text'})   

    # 빈 리스트 생성 - 수집한 각 data를 담을 그릇
    headline_title=[]
    headline_link=[]
    headline_desc=[]
    
    # 각 headline 기사의 세부내용 추출 후 list에 추가코드
    for head in head_line :
        try : 
            a_t = head.find('a')
            div_t = head.find('div',{'class':"sh_text_lede"})
            headline_title.append(a_t.text)
            headline_link.append(a_t['href'])
            headline_desc.append(div_t.text)
        except :
            print('에러발생')
            
    # 수집된 data를 dict로 생성 후 반환
    return({'기사제목':headline_title,
     '기사링크':headline_link,
     '기사요약':headline_desc,'분류':section})

In [6]:
# test url 이용 함수 test
url = df_menu.loc[1].link

In [7]:
df_menu.loc[1]

section                                                   경제
link       https://news.naver.com/main/main.naver?mode=LS...
Name: 1, dtype: object

In [8]:
# 함수 테스트 후 df 생성
pd.DataFrame(get_topic(url,'경제'))

,기사제목,기사링크,기사요약,분류
0,"포스코인터 자회사 세넥스에너지, 호주에 10년간 천연가스 장기공급",https://n.news.naver.com/mnews/article/029/000...,포스코인터내셔널은 자회사인 호주 천연가스기업 세넥스에너지가 최근 장기 가스공급 계약...,경제
1,EU 공급망실사 지침에…금감원·인천시·하나금융 공동 중기 지원,https://n.news.naver.com/mnews/article/018/000...,이복현(사진) 금융감독원장은 10일 인천시 청라동 하나 글로벌캠퍼스에서 유정복 인천...,경제
2,"미래에셋증권, 상반기 순이익 3791억원…전년 대비 -20%",https://n.news.naver.com/mnews/article/421/000...,미래에셋증권(006800)이 상반기 지난해보다 20%가량 줄어든 순이익을 거뒀다. ...,경제
3,"“글로벌 숙박 플랫폼, 취소‧환불 지연 및 거부 사례 많아”",https://n.news.naver.com/mnews/article/366/000...,소비자원이 국내 소비자들이 많이 이용하는 글로벌 숙박 플랫폼들을 조사한 결과 부당한...,경제
4,"내수로 경기 버텼는데…고물가에 소비 급감, 제주 제일 아팠다",https://n.news.naver.com/mnews/article/025/000...,거리두기 해제 이후 회복세를 이어가던 소비가 꺾이는 모양새다. 올해 2분기 서비스업...,경제
5,"""고양이에 생선 맡겼네"" 명의도용에 횡령까지 '선 넘은 은행원들'",https://n.news.naver.com/mnews/article/008/000...,은행원의 '불법 일탈'이 끊이지 않고 있다. 고객 개인 정보를 몰래 이용해 실적 부...,경제
6,늘어나던 수도권 전세 신규계약…전셋값 회복에 4월 이후 비중↓,https://n.news.naver.com/mnews/article/001/001...,역전세 위험 낮아지자 전세계약 갱신 택하는 임차인 늘어난듯 올해 4월 이후 수도권 ...,경제
7,가계대출 늘었는데…주담대 금리 우상향에 차주 부담 늘어,https://n.news.naver.com/mnews/article/003/001...,은행권 가계대출 잔액이 사상 최대치를 기록하는 가운데 주택담보대출 금리가 우상향 흐...,경제
8,국세청장 “세무조사 최소화…고액·상습 체납 징수는 강화”,https://n.news.naver.com/mnews/article/032/000...,김창기 국세청장은 10일 “주요 세목의 신고·납부 상황을 치밀하게 관리하고 세수추이...,경제
9,"부영그룹, 용산철도고에 기숙사 지어 기증",https://n.news.naver.com/mnews/article/001/001...,부영그룹은 용산철도고등학교에 기숙사인 '우정학사'를 지어 기증했다고 10일 밝혔다....,경제


### 모든 섹션에 함수 적용 후 df로 생성 하고 csv 파일로 저장
0. 빈 df 생성 한 후에
1. 반복문을 통해 아래 작업 진행
    1. 반환되는 각 섹션의 dict를 temp df로 생성 후
    2. 기존 저장 df와 결합

In [9]:
# 0.빈 df 생성 한 후에
dict_sub={}
dict_sub['기사제목']=[]
dict_sub['기사링크']=[]
dict_sub['기사요약']=[]
dict_sub['분류']=[]
headline_df = pd.DataFrame(dict_sub)
headline_df

,기사제목,기사링크,기사요약,분류


In [10]:
# 1. 반복문을 통해 아래 작업 진행
#     1. 반환되는 각 섹션의 dict를 temp df로 생성 후
#     2. 기존 저장 df와 결합

# df_menu['link'] # 0번행부터~ 5번행까지
# df_menu['section']

for i in range(0,6) :
    temp = pd.DataFrame(get_topic(df_menu['link'][i], df_menu['section'][i]))
    headline_df = pd.concat([headline_df, temp], axis=0, ignore_index=True)

In [11]:
headline_df.head()
headline_df.tail()

,기사제목,기사링크,기사요약,분류
0,"[단독] 국방차관 문자 내역에 ""해병사령관에 보낸 건 '0'건""",https://n.news.naver.com/mnews/article/057/000...,고 채수근 상병 사건 조사 결과를 본 국방부 차관이 해병대 사령관에게 문자 메시지를...,정치
1,"국힘 “민주 혁신위, 용두사미 될 거란 예측 맞아떨어져”",https://n.news.naver.com/mnews/article/005/000...,국민의힘은 공천룰 개정 등 혁신안을 발표한 뒤 활동을 마감한 더불어민주당 혁신위원회...,정치
2,"檢, '백현동 개발 비리' 관련 이재명 대표 소환 조율",https://n.news.naver.com/mnews/article/079/000...,성남 백현동 개발 비리 의혹을 수사하는 검찰이 당시 성남시장으로 사업 인허가의 최종...,정치
3,"尹, “유엔사 의미 각별…국민들, 가짜평화 주장 안속아”",https://n.news.naver.com/mnews/article/081/000...,유엔사 주요직위자와 간담회 “반국가세력이 유엔사 해체 주장” 윤석열 대통령은 10일...,정치
4,재외동포청 경력경쟁채용 62명 선발… 경쟁률 15대 1,https://n.news.naver.com/mnews/article/421/000...,올 6월 공식 출범한 외교부 산하 재외동포청이 경력경쟁채용으로 총 62명의 직원을 ...,정치


,기사제목,기사링크,기사요약,분류
50,"불 르몽드 ""한 잼버리, 공금횡령 의혹까지 거론되며 빛바래""",https://n.news.naver.com/mnews/article/055/000...,폭염 속에 개막했다가 태풍의 위협으로 조기 철수한 2023 새만금 세계 스카우트 잼...,세계
51,"美, 첫 첨단기술 대중 투자 규제 발표…韓 동참 압박 우려",https://n.news.naver.com/mnews/article/449/000...,미국 정부가 중국 첨단산업에 대한 미국 자본의 투자를 막는 행정명령을 발표했습니다....,세계
52,"니제르 반군 지도자, 쿠데타 군부 퇴진 운동 전개…""바줌 대통령 복권 목표""",https://n.news.naver.com/mnews/article/421/000...,전직 니제르 반군 지도자가 9일(현지시간) 쿠데타 군부 퇴진 운동을 시작했다. 쿠데...,세계
53,WSJ “사우디·이스라엘 수교 큰 틀 합의”,https://n.news.naver.com/mnews/article/022/000...,9개월∼1년 내 관계 정상화 전망 재선 도전 바이든에 호재 될 수도 ‘팔레스타인 독...,세계
54,"美반도체법 1년…바이든 ""218조 원대 기업 투자 성과""(종합)",https://n.news.naver.com/mnews/article/003/001...,김난영 특파원 = 조 바이든 미국 대통령이 반도체법(CHIPs Act) 제정 1년을...,세계


In [12]:
# 파일로 저장
headline_df.to_csv('./crawl_data/naver_news_headline.csv')